# Verification and Hyperparameters

## 1. Model evaluation

### 1.01 Time-delay neural net predictions

In [1]:
import numpy as np
import datetime as dt
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr

import sys
sys.path.append('../../')
from python.aux.plot import plot_recurrent

In [9]:
# load the truth (reanalysis)
glofas = xr.open_dataset('../../data/smallsampledata-glofas.nc')['dis']

lat, lon = 48.35, 13.95  # the point where predictions are made
truth = glofas.interp(latitude=lat, longitude=lon)
truth

<xarray.DataArray 'dis' (time: 5478)>
array([798.361572, 792.207703, 796.706116, ..., 783.30896 , 782.144958,
       778.828003])
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 1995-12-31
    latitude   float64 48.35
    longitude  float64 13.95
Attributes:
    long_name:  discharge
    units:      m3/s

In [10]:
# load predictions
model_dir = '../../models/tdnn-catchment_mean/'

validation = test = xr.open_dataset(model_dir+'validation.nc')
test = xr.open_dataset(model_dir+'test.nc').to_array().drop('variable')
test

<xarray.DataArray (variable: 1, init_time: 717, forecast_day: 14)>
array([[[1087.5364 , 1116.3853 , ..., 1183.2561 , 1201.5026 ],
        [1088.2441 , 1124.8561 , ..., 1103.9218 , 1151.1743 ],
        ...,
        [ 789.5235 ,  807.2123 , ...,  838.81226,  785.8096 ],
        [ 687.283  ,  782.11707, ...,  776.7037 ,  750.24243]]], dtype=float32)
Coordinates:
  * init_time     (init_time) datetime64[ns] 1993-12-31T23:00:00 ... 1995-12-17T23:00:00
  * forecast_day  (forecast_day) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14
    name          float64 ...
Dimensions without coordinates: variable

In [ ]:
test_time = slice('1994', '1995')

fix, ax = plt.subplots(figsize=(15,5))
plot_recurrent(ax, truth=truth.sel(time=test_time), 
                   prediction=test.sel(init_time=test_time))


time=slice('1994-03', '1994-05')

fix, ax = plt.subplots(figsize=(15,5))
plot_recurrent(ax, truth=truth.sel(time=time), 
                   prediction=test.sel(init_time=time))

### 1.01 Time-delay neural net predictions

## 2. Hyperparameters

### 2.01 Time-delay neural net

- Architecture (hidden layers, number of nodes)

description




### 2.02 LSTM network

- Architecture (stacked layers)

description

- Number of forecast days


- Regularization (dropout)

description